## Spacy NER Training
### For a complete guide follow the link: https://spacy.io/usage/training#ner

In [3]:
import spacy
from spacy.util import minibatch, compounding
from pathlib import Path
import random

In [4]:
nlp = spacy.load("en_core_web_sm")


### Training Data

In [14]:
# training data
TRAIN_DATA = [
    ("Who is Shaka Khan?", {"entities": [(7, 17, "PERSON")]}),
    ("I like London and Berlin.", {"entities": [(7, 13, "LOC"), (18, 24, "LOC")]}),
]
TRAIN_DATA


[('Who is Shaka Khan?', {'entities': [(7, 17, 'PERSON')]}),
 ('I like London and Berlin.',
  {'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]})]

### Create the built-in pipeline components and add them to the pipeline


In [19]:
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe("ner")
    

In [20]:
# add labels
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

['tagger', 'parser', 'ner']

### Train NER 

In [21]:
n_iter=100
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):  # only train NER
    # reset and initialize the weights randomly – but only if we're
    # training a new model
    #if model is None:
    nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts,  # batch of texts
                annotations,  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                losses=losses,
            )
        print("Losses", losses)
        

Losses {'ner': 2.8372888102996097e-06}
Losses {'ner': 0.0012294779650008467}
Losses {'ner': 7.34081300397202e-08}
Losses {'ner': 0.20019094922002262}
Losses {'ner': 2.4767087001852048e-05}
Losses {'ner': 5.083150227132131e-12}
Losses {'ner': 9.118843736945031e-12}
Losses {'ner': 5.147040028196106e-11}
Losses {'ner': 7.75932161337328e-11}
Losses {'ner': 5.562305359233314e-08}
Losses {'ner': 4.737231688075041e-09}
Losses {'ner': 6.373910943043833e-08}
Losses {'ner': 5.349695386819556e-06}
Losses {'ner': 4.925292936694349e-10}
Losses {'ner': 7.962457511631027e-11}
Losses {'ner': 2.485504100240832e-06}
Losses {'ner': 5.305104124023901e-09}
Losses {'ner': 6.160425470939393e-07}
Losses {'ner': 5.762340055456883e-08}
Losses {'ner': 2.279735808642763e-11}
Losses {'ner': 1.7117894752192997e-07}
Losses {'ner': 1.0181217051071292e-10}
Losses {'ner': 2.5560355309692765e-12}
Losses {'ner': 1.6729238359921844e-06}
Losses {'ner': 0.00046401482581980333}
Losses {'ner': 7.355611006897543e-13}
Losses {'

### Test the trained model

In [25]:
# test the trained model
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('London', 'LOC'), ('Berlin', 'LOC')]
Tokens [('I', '', 2), ('like', '', 2), ('London', 'LOC', 3), ('and', '', 2), ('Berlin', 'LOC', 3), ('.', '', 2)]
Entities [('Shaka Khan', 'PERSON')]
Tokens [('Who', '', 2), ('is', '', 2), ('Shaka', 'PERSON', 3), ('Khan', 'PERSON', 1), ('?', '', 2)]


In [27]:
TRAIN_DATA = [('what is the price of polo?', {'entities': [(21, 25, 'PrdName')]}), 
              ('what is the price of ball?', {'entities': [(21, 25, 'PrdName')]}), 
              ('what is the price of jegging?', {'entities': [(21, 28, 'PrdName')]}), 
              ('what is the price of t-shirt?', {'entities': [(21, 28, 'PrdName')]}), 
              ('what is the price of jeans?', {'entities': [(21, 26, 'PrdName')]}), 
              ('what is the price of bat?', {'entities': [(21, 24, 'PrdName')]}), 
              ('what is the price of shirt?', {'entities': [(21, 26, 'PrdName')]}), 
              ('what is the price of bag?', {'entities': [(21, 24, 'PrdName')]}), 
              ('what is the price of cup?', {'entities': [(21, 24, 'PrdName')]}), 
              ('what is the price of jug?', {'entities': [(21, 24, 'PrdName')]}), 
              ('what is the price of plate?', {'entities': [(21, 26, 'PrdName')]}), 
              ('what is the price of glass?', {'entities': [(21, 26, 'PrdName')]}), 
              ('what is the price of moniter?', {'entities': [(21, 28, 'PrdName')]}), 
              ('what is the price of desktop?', {'entities': [(21, 28, 'PrdName')]}), 
              ('what is the price of bottle?', {'entities': [(21, 27, 'PrdName')]}), 
              ('what is the price of mouse?', {'entities': [(21, 26, 'PrdName')]}), 
              ('what is the price of keyboad?', {'entities': [(21, 28, 'PrdName')]}), 
              ('what is the price of chair?', {'entities': [(21, 26, 'PrdName')]}), 
              ('what is the price of table?', {'entities': [(21, 26, 'PrdName')]}), 
              ('what is the price of watch?', {'entities': [(21, 26, 'PrdName')]})]


def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp


prdnlp = train_spacy(TRAIN_DATA, 20)

# Save our trained Model
modelfile = input("Enter your Model Name: ")
prdnlp.to_disk(modelfile)

#Test your text
test_text = input("Enter your testing text: ")
doc = prdnlp(test_text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


Statring iteration 0
{'ner': 47.07635166372347}
Statring iteration 1
{'ner': 3.310417786359703}
Statring iteration 2
{'ner': 1.99949655747491}
Statring iteration 3
{'ner': 1.4026618689367127}
Statring iteration 4
{'ner': 1.676383895302153}
Statring iteration 5
{'ner': 6.4542502900135545}
Statring iteration 6
{'ner': 3.7191266373405756}
Statring iteration 7
{'ner': 2.2737538025556354}
Statring iteration 8
{'ner': 5.482526095537221}
Statring iteration 9
{'ner': 0.7795053744254973}
Statring iteration 10
{'ner': 2.819928162995283}
Statring iteration 11
{'ner': 2.2508015325621544}
Statring iteration 12
{'ner': 1.6131304780371398}
Statring iteration 13
{'ner': 0.15742579413160138}
Statring iteration 14
{'ner': 1.9461783597948787}
Statring iteration 15
{'ner': 0.013275587068445376}
Statring iteration 16
{'ner': 6.915177632522351e-05}
Statring iteration 17
{'ner': 2.1704969131956125e-06}
Statring iteration 18
{'ner': 6.605839161582747e-06}
Statring iteration 19
{'ner': 1.414577497481693e-07}
E